# Prompt Engineering in LangChain

##1. Installation and Setup

In [1]:
!pip install -U langchain langchain-openai langchain_community

##2. Setup and Model Initialization
In this section, we'll set up our environment and initialize two language model instances with different temperature settings. Understanding temperature is crucial for prompt engineering:

- Temperature controls the randomness of the model's output by affecting how it selects the next token (word or word-piece). According to OpenAI's guidelines, there are three main categories:

    - **Low-temperature values (0.0 to 0.8)**: Use for analytical, factual, or logical tasks where the model should be more deterministic and focused. Lower temperatures help the model adhere to established patterns and conventions, leading to more correct and repeatable answers. Examples: generating code, performing data analysis, answering factual questions.

    - **Medium-temperature values (0.8 to 1.2)**: Use for general-purpose and chatbot-like tasks where balancing coherence and creativity is critical. This enables the model to be flexible and produce new ideas while remaining focused on the prompt.

    - **High-temperature values (1.2 to 2.0)**: Use for creative writing and brainstorming where the model should explore diverse styles and unexpected outputs. A correct answer may not exist—the purpose is to create varying outputs. Examples: storytelling, generating marketing slogans, brainstorming company names.

For this notebook, we'll use 0.2 (low) and 1.5 (high) to demonstrate the extremes of this spectrum.

In [2]:
from google.colab import userdata

# Import the necessary libraries from LangChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Retrieve your OpenAI API key from Colab's secure storage
# Note: You'll need to add your API key to Colab Secrets before running this
API_KEY = userdata.get('OPENAI_API_KEY')

# Define temperature values for our two model instances
low_temp = 0.2    # Low temperature: deterministic, analytical responses
high_temp = 1.5   # High temperature: creative, diverse responses

# Initialize the low-temperature model
# Use this for tasks requiring accuracy, consistency, and adherence to patterns
model_low = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=API_KEY,
    temperature=low_temp,
    max_tokens=1000  # Maximum length of the response
)

# Initialize the high-temperature model
# Use this for tasks requiring creativity, exploration, and varied outputs
model_high = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=API_KEY,
    temperature=high_temp,
    max_tokens=1000
)

print(f"LangChain models (gpt-4o-mini) initialized: one with temperature={low_temp} and one with temperature={high_temp}.")

LangChain models (gpt-4o-mini) initialized: one with temperature=0.2 and one with temperature=1.5.


##3. Anatomy of a Prompt
Before we dive into prompt engineering techniques, it's important to understand the building blocks of an effective prompt. A well-constructed prompt typically has four components:
- **Input**: The raw query or variable data from the user (e.g., a topic, a question, a piece of text to analyze)
- **Instruction**: Clear guidelines or directives that tell the model what task to perform
- **Context (Optional)**: Additional information that helps ground the response—such as examples, business data, background information, or retrieved content from a database
- **System Prompt (Optional)**: Persistent instructions that define the model's role, tone, or behavioral constraints (e.g., "You are a helpful assistant" or "You are an expert data scientist")

Understanding these components will help you design more effective prompts. Let's start with a simple example that uses only Input and Instruction.

###3.1 Simple Prompt Example
In this first example, we'll create a basic prompt that asks the model to generate a business story. Notice that we're only using two of the four components:

- **Input**: The topic (provided by the variable `topic`)
- **Instruction**: The task description embedded in the template

In [3]:
# Create a simple prompt template with just Input and Instruction
simple_prompt = PromptTemplate(
    input_variables=["topic"],  # The 'topic' variable represents user-supplied input
    template="Tell me a story about how {topic} can transform the business operations of a sports medicine firm."
    # The template string is the Instruction—it tells the model exactly what to do
)

# Define the input value
select_topic = "large language models"

# Display the prompt template structure
print("Simple Prompt:")
print(simple_prompt.template)

# Run the prompt through both temperature settings to compare outputs
print(f"\nResponse with low temperature ({low_temp}):")
low_chain = simple_prompt | model_low  # Create a chain: prompt → model
print(low_chain.invoke({"topic": select_topic}).content)

print(f"\nResponse with high temperature ({high_temp}):")
high_chain = simple_prompt | model_high  # Same prompt, different temperature
print(high_chain.invoke({"topic": select_topic}).content)

Simple Prompt:
Tell me a story about how {topic} can transform the business operations of a sports medicine firm.

Response with low temperature (0.2):
**Title: The Game Changer: Transforming Sports Medicine with AI**

In the bustling city of Athletica, a renowned sports medicine firm called Peak Performance was struggling to keep up with the demands of its growing clientele. Established by a group of passionate sports physicians and physical therapists, the firm had built a solid reputation for helping athletes recover from injuries and enhance their performance. However, as the number of clients increased, so did the challenges of managing patient data, treatment plans, and communication.

One day, the firm’s co-founder, Dr. Sarah Chen, attended a technology conference where she learned about the potential of large language models (LLMs) in various industries. Intrigued by the possibilities, she returned to Peak Performance with a vision: to integrate LLMs into their operations to st

What to observe:

- The low-temperature response should be more structured and straightforward
- The high-temperature response may include more creative narrative elements or unexpected angles
- Both responses address the same instruction, but with different levels of creativity

###3.2 Extended Prompt Example with Context and System Prompt
Now let's see how adding Context and a System Prompt can significantly improve and customize the model's output. This example demonstrates all four prompt components:

- **Input**: The topic variable (e.g., "large language models")
- **Instruction**: The explicit task directive ("Give me a business rundown...")
- **Context**: Specific business background that grounds the response in your company's situation
- **System Prompt**: A role definition that shapes the tone and expertise level of the response

By including these additional components, we can generate responses that are more relevant, targeted, and aligned with our specific needs.

In [4]:
# Create an extended prompt template using all four components
extended_prompt = PromptTemplate(
    input_variables=["topic", "context", "system_prompt"],
    template=(
        "{system_prompt}\n"              # System Prompt: Defines the model's role and expertise
        "Context: {context}\n"           # Context: Provides relevant business background
        "Instruction: Give me a business rundown on how {topic} can transform the business operations of a sports medicine firm."
        # Instruction: The specific task we want the model to perform
    )
)

# Define values for all three input variables
select_topic = "large language models"
select_context = "Our company is planning to integrate advanced AI solutions into clinical and operational processes to enhance patient care and streamline workflows."
select_system_prompt = "You are a seasoned business consultant with deep expertise in digital transformation in the healthcare industry."

# Display the full prompt structure
print("Extended Prompt with Context and System Prompt:")
print(extended_prompt.template)

# Run the extended prompt through both temperature settings
print(f"\nResponse with low temperature ({low_temp}):")
low_chain = extended_prompt | model_low
print(low_chain.invoke({
    "topic": select_topic,
    "context": select_context,
    "system_prompt": select_system_prompt
}).content)

print(f"\nResponse with high temperature ({high_temp}):")
high_chain = extended_prompt | model_high
print(high_chain.invoke({
    "topic": select_topic,
    "context": select_context,
    "system_prompt": select_system_prompt
}).content)

Extended Prompt with Context and System Prompt:
{system_prompt}
Context: {context}
Instruction: Give me a business rundown on how {topic} can transform the business operations of a sports medicine firm.

Response with low temperature (0.2):
Integrating advanced AI solutions, particularly large language models (LLMs), into the operations of a sports medicine firm can significantly enhance both clinical and operational processes. Here’s a comprehensive business rundown on how LLMs can transform your firm:

### 1. **Enhanced Patient Engagement and Communication**
   - **Personalized Communication**: LLMs can analyze patient data and preferences to generate personalized messages, reminders, and educational content, improving patient engagement and adherence to treatment plans.
   - **24/7 Virtual Assistants**: Implementing AI-driven chatbots can provide patients with instant responses to common inquiries, appointment scheduling, and follow-up care instructions, reducing the burden on admin

What to observe:

- Compare these responses to the simple prompt outputs—notice how much more specific and relevant they are
- The System Prompt creates an "expert consultant" persona that affects word choice and depth
- The Context ensures the response addresses your company's specific situation
- Even with high temperature, the response should stay focused for longer because of the strong context and system prompt

Key Takeaway: Adding Context and System Prompts dramatically improves output quality and relevance. Always consider which of the four components your task requires.

##4. One-Shot Learning: Customer Service Email Generation
One-shot learning is a prompt engineering technique where you provide the model with a single example to demonstrate the desired output format, tone, and structure. This is particularly effective when you want the model to follow a specific template or style.
In this example, we'll use one-shot learning to generate professional customer service emails. By showing the model one example of a well-crafted response, we can guide it to produce similar outputs for different customer complaints.

### Anatomy of This One-Shot Prompt:

- **Input**: The scenario variable containing the specific customer complaint
- **Instruction**: "Compose a professional customer service email in response to a customer complaint"
- **Context**: The example email demonstrating the desired tone, structure, and professionalism (apology, explanation, closing)
- **System Prompt**: Not explicitly included here, but could be added to enforce consistent professional tone across all responses

The key insight: The example serves as the context that teaches the model what kind of response you want.

In [5]:
# Define the one-shot prompt template with an example email
one_shot_template = """
Compose a professional customer service email in response to a customer complaint.

Example:
Subject: Apology for the Delayed Shipment
Dear Valued Customer,
We sincerely apologize for the delay in your shipment. We are actively working to resolve the issue and ensure timely deliveries in the future. Please feel free to reach out with any further concerns.
Best regards,
Customer Service Team

Now, write a response for the following scenario:
Customer complaint: {scenario}
Email:
"""

# Create a PromptTemplate object with one input variable
one_shot_prompt = PromptTemplate(
    input_variables=["scenario"],  # Only the customer complaint varies; the example stays constant
    template=one_shot_template
)

# Define the specific customer complaint we want to respond to
select_scenario = "My order was delayed by a week, and I did not receive any update."

# Display the full prompt structure
print("One-Shot Business Email Prompt:")
print(one_shot_prompt.template)

# Run the prompt through both temperature settings
print(f"\nResponse with low temperature ({low_temp}):")
low_chain = one_shot_prompt | model_low
print(low_chain.invoke({"scenario": select_scenario}).content)

print(f"\nResponse with high temperature ({high_temp}):")
high_chain = one_shot_prompt | model_high
print(high_chain.invoke({"scenario": select_scenario}).content)

One-Shot Business Email Prompt:

Compose a professional customer service email in response to a customer complaint.

Example:
Subject: Apology for the Delayed Shipment
Dear Valued Customer,
We sincerely apologize for the delay in your shipment. We are actively working to resolve the issue and ensure timely deliveries in the future. Please feel free to reach out with any further concerns.
Best regards,
Customer Service Team

Now, write a response for the following scenario:
Customer complaint: {scenario}
Email:


Response with low temperature (0.2):
Subject: Apology for the Delay in Your Order

Dear [Customer's Name],

Thank you for reaching out to us regarding the delay in your order. We sincerely apologize for the inconvenience this has caused and understand how frustrating it can be to not receive timely updates.

We are currently investigating the issue to ensure that your order is shipped as soon as possible. Please rest assured that we are committed to improving our communication 

**What to observe:**
- Both outputs should follow the structure and tone of the example email
- The low-temperature version will likely stay closer to the example's wording and format
- The high-temperature version may introduce more variation while still maintaining professionalism
- Notice how the single example effectively "teaches" the model the desired output pattern

**When to use one-shot learning:**
- When you have a clear template or format to follow
- When you want consistent structure but varied content
- When zero-shot prompts (no examples) produce inconsistent results
- When you don't need multiple examples to convey the pattern

Key Takeaway: One-shot learning is efficient and a single well-crafted example can be enough to guide the model toward your desired output format and style.

## 5. Few-Shot Learning: Sentiment Analysis on Product Reviews
Few-shot learning extends the one-shot approach by providing multiple examples to help the model understand patterns and produce consistent outputs. This technique is especially powerful for classification tasks where you want the model to learn from several demonstrations.In this example, we'll use few-shot learning to classify customer reviews as positive or negative. By providing four examples (two positive, two negative), we teach the model both the task and the exact output format we expect.
### Anatomy of This Few-Shot Prompt:
- **Input**: The `new_review` variable containing a customer review to classify
- **Instruction**: "Determine the sentiment (positive/negative) for the following customer review"
- **Context**: Four example reviews with their sentiment labels—this is where the learning happens
- **System Prompt**: Not explicitly set here, but could be added to maintain business tone

Important: Notice that we embed the examples directly into the template string using an f-string. The examples are not passed as a separate input variable because they're baked into the prompt itself.

In [6]:
# Define a list of few-shot examples showing both positive and negative sentiments
# These examples serve as the Context component of our prompt
few_shot_examples = [
    "Review: 'The product quality exceeded my expectations.' -> Sentiment: positive",
    "Review: 'Delivery was late and the packaging was poor.' -> Sentiment: negative",
    "Review: 'Customer service was very helpful.' -> Sentiment: positive",
    "Review: 'The item did not match the description.' -> Sentiment: negative"
]

# Create a template that combines Instruction, Context, and Input
# Note: We use an f-string to embed the examples, and {{new_review}} (double braces)
# to preserve the placeholder for LangChain's template system
few_shot_template = f"""
Determine the sentiment (positive/negative) for the following customer review.

Examples:
{few_shot_examples}

New Review: {{new_review}}
Sentiment:
"""

# Create a PromptTemplate instance with only one input variable
# The examples are already embedded in the template, not passed as a variable
few_shot_prompt = PromptTemplate(
    input_variables=["new_review"],  # Only the review to classify is dynamic
    template=few_shot_template
)

# Define a new review to classify
select_new_review = "The checkout process was smooth and the support team responded quickly."

# Display the full prompt structure
print("Few-Shot Business Prompt:")
print(few_shot_prompt.template)

# Run the prompt through both temperature settings
print(f"\nResponse with low temperature ({low_temp}):")
low_chain = few_shot_prompt | model_low
print(low_chain.invoke({"new_review": select_new_review}).content)

print(f"\nResponse with high temperature ({high_temp}):")
high_chain = few_shot_prompt | model_high
print(high_chain.invoke({"new_review": select_new_review}).content)

Few-Shot Business Prompt:

Determine the sentiment (positive/negative) for the following customer review.

Examples:
["Review: 'The product quality exceeded my expectations.' -> Sentiment: positive", "Review: 'Delivery was late and the packaging was poor.' -> Sentiment: negative", "Review: 'Customer service was very helpful.' -> Sentiment: positive", "Review: 'The item did not match the description.' -> Sentiment: negative"]

New Review: {new_review}
Sentiment:


Response with low temperature (0.2):
Sentiment: positive

Response with high temperature (1.5):
Sentiment: positive


**What to observe:**
- Both models should correctly classify the sentiment as "positive"
- The output format should match the examples: just "positive" or "negative"
- Low temperature should give you consistent, reliable classifications
- High temperature might occasionally produce unexpected outputs or additional commentary
- The four examples teach the model both what to classify AND how to format the answer

**Few-Shot vs. One-Shot:**
- One-shot: Use when the pattern is simple and one example suffices
- Few-shot: Use when you need to show multiple variations or edge cases
- General rule: More examples = more consistent output, but also longer prompts (and higher costs)

**When to use few-shot learning:**
- Classification tasks (sentiment, category, intent, etc.)
- When you need consistent output formatting
- When the task requires understanding multiple patterns or categories
- When one-shot doesn't provide enough guidance

Key Takeaway: Few-shot learning is particularly effective for classification and structured output tasks. The examples act as training data that guide the model toward your desired behavior.

##6. Structured Output: Controlling Response Format
So far, we've focused on crafting effective prompts using different learning strategies (zero-shot, one-shot, few-shot). But what about controlling the format of the output itself?

Sometimes you need responses in a specific structure—like JSON, XML, or a particular text format. This is crucial when integrating LLM outputs into downstream applications, databases, or APIs.

In this section, we'll explore two approaches:
- **Prompt-based formatting**: Using instructions and examples to request structured output
- **Constrained output with OpenAI's response format parameter**: Guaranteeing JSON output

###6.1 Approach 1: Zero-Shot Structured Output (Prompt-Based)
Let's start by simply asking the model to generate JSON without providing any examples. This tests the model's ability to infer structure from the instruction alone.

In [7]:
# Zero-shot: Request JSON format without providing examples
zero_shot_json_prompt = "Create a character profile for an RPG game in JSON format."

print("Zero-Shot JSON Request:")
print(zero_shot_json_prompt)

# Test with low temperature for more consistent formatting
print(f"\nResponse with low temperature ({low_temp}):")
print(model_low.invoke(zero_shot_json_prompt).content)

print(f"\nResponse with high temperature ({high_temp}):")
print(model_high.invoke(zero_shot_json_prompt).content)

Zero-Shot JSON Request:
Create a character profile for an RPG game in JSON format.

Response with low temperature (0.2):
Here's a character profile for an RPG game in JSON format:

```json
{
  "character": {
    "name": "Elysia Windrider",
    "race": "Elf",
    "class": "Ranger",
    "level": 5,
    "attributes": {
      "strength": 12,
      "dexterity": 18,
      "constitution": 14,
      "intelligence": 15,
      "wisdom": 17,
      "charisma": 10
    },
    "skills": {
      "archery": 5,
      "stealth": 4,
      "survival": 5,
      "animalHandling": 3,
      "perception": 4
    },
    "equipment": {
      "weapons": [
        {
          "name": "Longbow",
          "damage": "1d8",
          "range": "150/600"
        },
        {
          "name": "Dagger",
          "damage": "1d4",
          "range": "20/60"
        }
      ],
      "armor": {
        "type": "Leather Armor",
        "armorClass": 12
      },
      "items": [
        "Healing Potion",
        "Rations (5 da

**What to observe:**
- The model likely produces valid JSON, but the exact structure may vary between runs
- High temperature may introduce more creative fields or inconsistent formatting
- Without examples, you have less control over which fields appear

###6.2 Approach 2: One-Shot Structured Output (Template-Based)
Now let's provide a concrete example of the JSON structure we want. This gives the model a clear template to follow.

In [8]:
# One-shot: Provide a template showing the exact JSON structure desired
one_shot_json_template = """Create a short character profile for an RPG game. Make sure to only use this format:

{
  "name": "THE CHARACTER'S NAME",
  "description": "A SHORT DESCRIPTION",
  "armor": "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}
"""

print("One-Shot JSON Template:")
print(one_shot_json_template)

# Test with low temperature for consistent structure adherence
print(f"\nResponse with low temperature ({low_temp}):")
print(model_low.invoke(one_shot_json_template).content)

print(f"\nResponse with high temperature ({high_temp}):")
print(model_high.invoke(one_shot_json_template).content)

One-Shot JSON Template:
Create a short character profile for an RPG game. Make sure to only use this format:

{
  "name": "THE CHARACTER'S NAME",
  "description": "A SHORT DESCRIPTION",
  "armor": "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}


Response with low temperature (0.2):
{
  "name": "Elysia Windrider",
  "description": "A skilled elven ranger with a deep connection to nature, Elysia is known for her sharp wit and unparalleled archery skills. She roams the forests, protecting the realm from dark forces.",
  "armor": "Leaf-patterned leather armor",
  "weapon": "Elven longbow and a pair of enchanted daggers"
}

Response with high temperature (1.5):
{
  "name": "Lyra Nightwhisper",
  "description": "A cunning rogue with a mysterious past, Lyra excels in stealth and deception. She's known for her quick wit and a charming smile that masks her deadly proficiency in combat.",
  "armor": "Shadow Cloak of Elusiveness",
  "weapon": "Dual daggers and a collapsible crossbow"
}

###6.3 Approach 3: Guaranteed JSON with OpenAI's Response Format
OpenAI's API offers a `response_format` parameter that guarantees valid JSON output. This uses constrained sampling to ensure the model only generates tokens that form valid JSON.

In [9]:
# Initialize a model instance with JSON response format enforced
model_json = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=API_KEY,
    temperature=0,  # Use 0 for maximum consistency
    max_tokens=1000,
    model_kwargs={"response_format": {"type": "json_object"}}  # Enforce JSON output
)

# Simple prompt - the model will automatically format as JSON
json_prompt = """Create an RPG character with the following JSON structure:

{
  "name": "character name",
  "class": "character class",
  "level": numeric level,
  "stats": {
    "strength": number,
    "dexterity": number,
    "intelligence": number
  },
  "equipment": {
    "weapon": "weapon name",
    "armor": "armor name"
  }
}

Generate a warrior character."""

print("Guaranteed JSON Output (using response_format parameter):")
response = model_json.invoke(json_prompt).content
print(response)

Guaranteed JSON Output (using response_format parameter):
{
  "name": "Thorin Ironfist",
  "class": "Warrior",
  "level": 5,
  "stats": {
    "strength": 18,
    "dexterity": 12,
    "intelligence": 8
  },
  "equipment": {
    "weapon": "Battle Axe",
    "armor": "Plate Armor"
  }
}


In [10]:
# Verify it's valid JSON by parsing it
import json
try:
    parsed_json = json.loads(response)
    print("\n✓ Valid JSON confirmed!")
    print("\nFormatted output:")
    print(json.dumps(parsed_json, indent=2))
except json.JSONDecodeError:
    print("\n✗ JSON parsing failed")


✓ Valid JSON confirmed!

Formatted output:
{
  "name": "Thorin Ironfist",
  "class": "Warrior",
  "level": 5,
  "stats": {
    "strength": 18,
    "dexterity": 12,
    "intelligence": 8
  },
  "equipment": {
    "weapon": "Battle Axe",
    "armor": "Plate Armor"
  }
}


**What to observe:**
- The output is guaranteed to be valid JSON
- You still need to prompt for the specific fields you want
- The model cannot produce non-JSON output even if it wants to
- This is the most reliable approach for production systems

**Key Takeaway:** For structured outputs in production systems, always use OpenAI's response_format parameter combined with clear schema instructions in your prompt. This gives you both validity guarantees and control over the exact structure.

## 7. Chain-of-Thought (CoT) Prompting: Improving Reasoning Through Intermediate Steps

Chain-of-Thought (CoT) prompting is a technique that encourages language models to break down complex problems into intermediate reasoning steps before arriving at a final answer. Research has shown that this approach significantly improves performance on tasks requiring logic, arithmetic, and multi-step reasoning.Why Chain-of-Thought MattersWhen models jump directly to answers (like humans doing mental math), they're more prone to errors. By explicitly asking the model to "show its work," we can:
- Improve accuracy on mathematical and logical tasks
- Make the reasoning process transparent and verifiable
- Catch errors in intermediate steps
- Build trust by understanding how the model arrived at its conclusion

Let's compare direct prompting (non-CoT) with Chain-of-Thought prompting using a business discount calculation.

###7.1 Non-Chain-of-Thought (Non-CoT) Example: Direct Answer
First, let's see what happens when we ask for a direct answer without requesting intermediate steps.
### Anatomy of this prompt:
- **Input**: Product price ($250) and discount percentage (20%)
- **Instruction**: "What is the final price? Don't think, just answer."
- **Context**: None
- **System Prompt**: Not explicitly defined

In [11]:
# Non-CoT prompt: Request a direct answer without showing work
non_cot_prompt = (
    "Our product is priced at $250. We want to offer a 20% discount to our loyal customers. "
    "What is the final price? Don't think, just answer."
)

print("Non-Chain-of-Thought Business Prompt:")
print(non_cot_prompt)

print(f"\nResponse with low temperature ({low_temp}):")
print(model_low.invoke(non_cot_prompt).content)

print(f"\nResponse with high temperature ({high_temp}):")
print(model_high.invoke(non_cot_prompt).content)

Non-Chain-of-Thought Business Prompt:
Our product is priced at $250. We want to offer a 20% discount to our loyal customers. What is the final price? Don't think, just answer.

Response with low temperature (0.2):
The final price after a 20% discount on $250 is $200.

Response with high temperature (1.5):
The final price after a 20% discount is $200.


**What to observe:**
- The model likely provides the correct answer ($200)
- However, you cannot verify the reasoning process
- For more complex calculations, this approach is more error-prone
- No transparency into how the answer was derived

###7.2 Chain-of-Thought (CoT) Example: Step-by-Step Reasoning
Now let's modify the prompt to explicitly request intermediate calculation steps. We'll also add an instruction to encourage deliberate, careful reasoning.
### Anatomy of this prompt:
- **Input**: Same product pricing details ($250, 20% discount)
- **Instruction**: "Show step-by-step calculations" + "Stop and think for 20 seconds before you answer each step"
- **Context**: The instruction to show steps creates implicit context that guides the reasoning process
- **System Prompt**: Not explicitly defined

Note: The phrase "Stop and think for 20 seconds" is a prompt engineering technique that encourages the model to engage in more deliberate, careful reasoning (similar to how asking a human to "slow down and think carefully" improves accuracy).

In [12]:
# CoT prompt: Request explicit step-by-step reasoning with deliberate pacing
cot_prompt = (
    "Our product is priced at $250. We want to offer a 20% discount to our loyal customers. "
    "Show step-by-step calculations to find the final price. Stop and think for 20 seconds before you answer each step."
)

print("\nChain-of-Thought Business Prompt:")
print(cot_prompt)

print(f"\nResponse with low temperature ({low_temp}):")
print(model_low.invoke(cot_prompt).content)

print(f"\nResponse with high temperature ({high_temp}):")
print(model_high.invoke(cot_prompt).content)


Chain-of-Thought Business Prompt:
Our product is priced at $250. We want to offer a 20% discount to our loyal customers. Show step-by-step calculations to find the final price. Stop and think for 20 seconds before you answer each step.

Response with low temperature (0.2):
Sure! Let's go through the calculations step-by-step to find the final price after applying a 20% discount to the original price of $250.

### Step 1: Calculate the amount of the discount.
To find the discount amount, we need to calculate 20% of the original price.

1. **Convert the percentage to a decimal**: 
   \[
   20\% = \frac{20}{100} = 0.20
   \]

2. **Multiply the original price by the decimal**:
   \[
   \text{Discount Amount} = 250 \times 0.20
   \]

Now, let's calculate that:
\[
250 \times 0.20 = 50
\]

So, the discount amount is **$50**.

### Step 2: Subtract the discount from the original price.
Now we will subtract the discount amount from the original price to find the final price.

1. **Subtract the 

**What to observe:**
- The model should break down the calculation into clear steps:
    1) Calculate the discount amount (20% of $250 = $50)
    2) Subtract discount from original price ($250 - $50 = $200)


- Each step is verifiable and transparent
- The reasoning process is more reliable than the direct answer approach
- Low temperature typically produces more consistent step formatting

**When to Use Chain-of-Thought Prompting**
- Use CoT for:
    - Mathematical calculations and word problems
    - Multi-step logical reasoning
    - Complex decision-making tasks
    - Situations where you need to verify the reasoning
    - Tasks where intermediate steps provide valuable insight
    - Instructional contexts (showing someone how to solve problems)

- Don't use CoT for:
    - Simple factual recall ("What is the capital of France?")
    - Creative writing tasks
    - When you need concise, quick responses
    - When the reasoning process isn't important